# Práctica 3:  Heteroscedasticidad y Autocorrelación

## Cuestiones

In [1]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
from wooldridge import *

1. Usando los datos de **sleep75**, analizar la posible existencia de heteroscedasticidad en el modelo $
sleep = \beta_0 + \beta_1 totwrk + \beta_2 educ + \beta_3 age + \beta_4 age^2 + \beta_5 yngkid + u$

2. Usando los datos de **vote1** para el modelo $voteA = \beta_0 + \beta_1 \log(expendA) + \beta_2 \log(expendB) + \beta_3 prtystrA + \beta_4 democA+ u$, analizar la heteroscedasticidad del modelo usando los Test de Breush-Pagan y White.

3. Con la base de datos **gpa1**:
 - Estimar $colGPA = \beta_0 + \beta_1 hsGPA + \beta_2 ACT + \beta_3 skipped + \beta_4 PC + u$ y extraer los residuos.
 - Contrastar la posibilidad de heteroscedasticidad con el test de Breush-Pagan.
 - Analizar, con el test de Glejser, la dependencia funcional de la heteroscedasticidad con respecto a la variables que podrían provocarla.
 - Ajustar, según el método de Mínimos Cuadrados Ponderados, el modelo con la corrección de heteroscedasticidad.

In [7]:
datos=dataWoo('barium', description=True)

name of dataset: barium
no of variables: 31
no of observations: 131

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| chnimp   | Chinese imports, bar. chl.      |
| bchlimp  | total imports bar. chl.         |
| befile6  | =1 for all 6 mos before filing  |
| affile6  | =1 for all 6 mos after filing   |
| afdec6   | =1 for all 6 mos after decision |
| befile12 | =1 all 12 mos before filing     |
| affile12 | =1 all 12 mos after filing      |
| afdec12  | =1 all 12 mos after decision    |
| chempi   | chemical production index       |
| gas      | gasoline production             |
| rtwex    | exchange rate index             |
| spr      | =1 for spring months            |
| sum      | =1 for summer months            |
| fall     | =1 for fall months              |
| lchnimp  | log(chnimp)                     |
| lgas     | log(gas)                        |
| lrtwex   | log(rtwex)               

4. Usando los datos de **barium**, estima el modelo $log(chempi)= \beta_0 + \beta_1 log(gas) + \beta_2 log(rtwex) + \beta_3 befile6  + \beta_4 affile6 + \beta_5  afdec6 + u$, y analiza la posible existencia de autocorrelación en el modelo. En caso afirmativo, estima el modelo corregido.

In [36]:
datos=dataWoo('barium')
y=datos['lchempi']
X=datos[['lgas', 'befile6', 'affile6', 'afdec6']]
mco4=sm.OLS(y, sm.add_constant(X)).fit()
mco4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lchempi   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     1.821
Date:                Tue, 03 Dec 2019   Prob (F-statistic):              0.129
Time:                        10:01:41   Log-Likelihood:                 103.71
No. Observations:                 131   AIC:                            -197.4
Df Residuals:                     126   BIC:                            -183.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9695      3.511     -1.131      0.260     -10.918       2.979
lgas           0.3836      0.154      2.499      0.014       0.080       0.687
befile6       -0.0275      0.047     -0.585      0.559      -0.120       0.065
affile6        0.0223      0.047      0.469      0.640      -0.072       0.116
afdec6         0.0600      0.048      1.239      0.217      -0.036       0.156
==============================================================================
Omnibus:                        3.775   Durbin-Watson:                   0.066
Prob(Omnibus):                  0.151   Jarque-Bera (JB):                2.550
Skew:                           0.160   Prob(JB):                        0.279
Kurtosis:                       2.396   Cond. No.                     8.24e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.24e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
from statsmodels.stats.stattools import durbin_watson
dw=durbin_watson(mco4.resid)

In [24]:
rho= 1 - dw/2


mco_autocorr=sm.GLSAR(y, sm.add_constant(X), rho=rho).iterative_fit(maxiter=1000)
mco_autocorr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           GLSAR Regression Results                           
==============================================================================
Dep. Variable:                lchempi   R-squared:                       0.008
Model:                          GLSAR   Adj. R-squared:                 -0.032
Method:                 Least Squares   F-statistic:                    0.2071
Date:                Tue, 03 Dec 2019   Prob (F-statistic):              0.959
Time:                        09:54:46   Log-Likelihood:                 363.67
No. Observations:                 130   AIC:                            -715.3
Df Residuals:                     124   BIC:                            -698.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3066      0.635      8.362      0.000       4.051       6.563
lgas          -0.0185      0.024     -0.780      0.437      -0.065       0.028
lrtwex         0.0050      0.055      0.091      0.928      -0.104       0.114
befile6       -0.0032      0.011     -0.295      0.768      -0.025       0.018
affile6       -0.0057      0.011     -0.528      0.599      -0.027       0.016
afdec6         0.0006      0.011      0.051      0.959      -0.021       0.022
==============================================================================
Omnibus:                       17.718   Durbin-Watson:                   1.092
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.729
Skew:                          -0.705   Prob(JB):                     2.59e-06
Kurtosis:                       4.662   Cond. No.                         385.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
np.mean(datos['afdec6'])

0.04580152671755725

5. Usando los datos de **prminwge**, estima el modelo lineal $\log( prepopt) = \beta_0 + \beta_1 \log(mincovt) + \beta_2 \log(usgnpt) + u$, y analiza la posible autocorrelación del modelo con el contraste de Ljung-Box, para distintos retardos.

In [45]:
dataWoo('prminwge', description=True)

name of dataset: prminwge
no of variables: 25
no of observations: 38

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| year     | 1950-1987                       |
| avgmin   | weighted avg min wge, 44 indust |
| avgwage  | wghted avg hrly wge, 44 indust  |
| kaitz    | Kaitz min wage index            |
| avgcov   | wghted avg coverage, 8 indust   |
| covt     | economy-wide coverage of min wg |
| mfgwage  | avg manuf. wage                 |
| prdef    | Puerto Rican price deflator     |
| prepop   | PR employ/popul ratio           |
| prepopf  | PR employ/popul ratio, alter.   |
| prgnp    | PR GNP                          |
| prunemp  | PR unemployment rate            |
| usgnp    | US GNP                          |
| t        | time trend:  1 to 38            |
| post74   | time trend:  starts in 1974     |
| lprunemp | log(prunemp)                    |
| lprgnp   | log(prgnp)              

In [47]:
datos=dataWoo('prminwge')

y=datos['lprepop']
X=datos[['lmincov', 'lusgnp']]
        
mco5=sm.OLS(y, sm.add_constant(X)).fit()
mco5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lprepop   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.641
Method:                 Least Squares   F-statistic:                     34.04
Date:                Tue, 03 Dec 2019   Prob (F-statistic):           6.17e-09
Time:                        10:10:45   Log-Likelihood:                 57.376
No. Observations:                  38   AIC:                            -108.8
Df Residuals:                      35   BIC:                            -103.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0544      0.765     -1.378      0.177      -2.608       0.499
lmincov       -0.1544      0.065     -2.380      0.023      -0.286      -0.023
lusgnp        -0.0122      0.089     -0.138      0.891      -0.192       0.168
==============================================================================
Omnibus:                        0.079   Durbin-Watson:                   0.340
Prob(Omnibus):                  0.961   Jarque-Bera (JB):                0.084
Skew:                           0.073   Prob(JB):                        0.959
Kurtosis:                       2.822   Cond. No.                         676.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
from statsmodels.stats.diagnostic import acorr_ljungbox
acorr_ljungbox(mco5.resid, lags=3)

(array([27.94397293, 46.75091935, 57.88034635]),
 array([1.24879321e-07, 7.04963857e-11, 1.66711664e-12]))